In [4]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import sqlalchemy as sq
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


In [16]:
data = pd.read_csv(r'D:\Datascience\fullstackdsproject\notebooks\data\playground-series-s3e8\train.csv') #openpyxl for .xlsx files
data

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


we have got to know what are the X - independent n Y - dependent feature 

In [ ]:
# X=data.drop(labels=["price"],axis=1)
X = data.iloc[:,:-1]
X

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [ ]:
# y=data[["price"]]
y = data.iloc[:,-1:]
y

,price
0,13619
1,13387
2,2772
3,666
4,14453
...,...
193568,1130
193569,2874
193570,3036
193571,681


In [19]:
#drop the id column in the X 
X.drop('id', inplace=True, axis=1)
X

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In independent feature we have categorical value we need to seggregate and do the encoding for the categorical

In [21]:
X.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
x          float64
y          float64
z          float64
dtype: object

In [69]:
categorical_columns = X.select_dtypes(include=object).columns
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [70]:
numerical_columns = X.select_dtypes(exclude=object).columns
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

Setting up the pipeline here for the automatic scaling n imputer(handlling missing value)

In [71]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [72]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
num_pipeline = Pipeline(
    steps =[
        ('imputer', SimpleImputer()), #handle the missing value with the mean n median(outliers)
        ('scaler', StandardScaler()) #mean of 0 and standard deviation of 1 
)

In [74]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [76]:
cat_pipeline  = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')), #handle the missing value with the mode for category
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories]))
    ]
)

In [60]:
cat_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['Fair', 'Good', 'Very Good',
                                             'Premium', 'Ideal'],
                                            ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
                                            ['I1', 'SI2', 'SI1', 'VS2', 'VS1',
                                             'VVS2', 'VVS1', 'IF']]))])

we need to pass the categorical_columns and numerical_columns in column Transformer

In [77]:
preprocessor = ColumnTransformer(
    [
        ('num_pipeline', num_pipeline, numerical_columns),
     ("cat_pipeline", cat_pipeline, categorical_columns)
     ]
)

In [78]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')),
                                ('cat_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinalencoder',
                                                  OrdinalEncoder(categories=[['Fair',
                                                                              'Good',
                                                                              'Very '
                                                                              'Good',
                                                                              'Premium',
                                                                              'Ideal'],
                                                                             ['D',
                                                                              'E',
                                                                              'F',
                                                                              'G',
                                                                              'H',
                                                                              'I',
                                                                              'J'],
                                                                             ['I1',
                                                                              'SI2',
                                                                              'SI1',
                                                                              'VS2',
                                                                              'VS1',
                                                                              'VVS2',
                                                                              'VVS1',
                                                                              'IF']]))]),
                                 Index(['cut', 'color', 'clarity'], dtype='object'))])

train test split data - 70% train data, 30% test data

In [162]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(X, y, random_state=30, test_size=0.33)
X_train


,carat,cut,color,clarity,depth,table,x,y,z
72759,0.24,Ideal,E,VVS2,62.4,57.0,3.97,3.99,2.48
88788,1.21,Ideal,G,VVS1,61.3,56.0,6.87,6.87,4.21
62380,0.31,Good,F,VS1,59.5,58.0,4.39,4.41,2.61
10474,0.40,Premium,G,VS2,61.1,60.0,4.74,4.75,2.90
8061,1.37,Premium,J,SI1,60.6,60.0,7.14,7.20,4.34
...,...,...,...,...,...,...,...,...,...
66455,0.31,Ideal,E,SI1,61.8,56.0,4.31,4.35,2.68
46220,1.25,Ideal,G,SI2,62.0,56.0,6.88,6.95,4.28
98804,1.00,Good,G,SI1,63.5,56.0,6.29,6.37,4.02
48045,1.10,Ideal,G,VS1,59.9,60.0,6.68,6.77,4.01


In [163]:
X_test

,carat,cut,color,clarity,depth,table,x,y,z
70432,0.53,Premium,E,VS2,60.8,56.0,5.24,5.21,3.19
64839,0.71,Very Good,H,SI1,62.9,57.0,5.67,5.69,3.56
185316,0.30,Ideal,H,IF,62.1,57.0,4.27,4.29,2.66
84658,1.24,Premium,G,VS2,61.6,61.0,6.88,6.82,4.21
31953,0.36,Premium,E,VS1,60.4,58.0,4.60,4.63,2.80
...,...,...,...,...,...,...,...,...,...
58758,2.01,Premium,J,VS1,58.7,59.0,8.27,8.20,4.83
35221,0.35,Ideal,I,VS1,61.9,55.0,4.54,4.57,2.81
93030,0.42,Ideal,D,SI1,62.3,57.0,4.80,4.82,2.99
136437,0.52,Ideal,E,VVS2,61.7,55.0,5.19,5.16,3.20


Calling the preprocessor here

In [164]:
preprocessor.fit_transform(X_train)

array([[-1.19090289,  0.53807804, -0.1214869 , ...,  4.        ,
         1.        ,  5.        ],
       [ 0.90452915, -0.47949203, -0.64243082, ...,  4.        ,
         3.        ,  6.        ],
       [-1.03968614, -2.14460667,  0.39945702, ...,  1.        ,
         2.        ,  4.        ],
       ...,
       [ 0.45087892,  1.5556481 , -0.64243082, ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66690284, -1.7745812 ,  1.44134486, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25645739,  0.81559715, -0.1214869 , ...,  4.        ,
         3.        ,  2.        ]])

In [165]:
preprocessor.transform(X_test)

array([[-0.56443352, -0.94202387, -0.64243082, ...,  3.        ,
         1.        ,  3.        ],
       [-0.17559046,  1.00060988, -0.1214869 , ...,  2.        ,
         4.        ,  2.        ],
       [-1.06128854,  0.26055893, -0.1214869 , ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.80205983,  0.44557167, -0.1214869 , ...,  4.        ,
         0.        ,  2.        ],
       [-0.58603591, -0.10946655, -1.16337475, ...,  4.        ,
         1.        ,  5.        ],
       [-0.82366222, -0.10946655, -0.64243082, ...,  4.        ,
         3.        ,  4.        ]])

In [166]:
preprocessor.get_feature_names_out() #all features has been preprocessed and added with the pipeline name with the userdefined name

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [167]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())
X_train



,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.190903,0.538078,-0.121487,-1.573815,-1.570070,-1.527658,4.0,1.0,5.0
1,0.904529,-0.479492,-0.642431,1.039150,1.041460,0.977290,4.0,3.0,6.0
2,-1.039686,-2.144607,0.399457,-1.195386,-1.189222,-1.339424,1.0,2.0,4.0
3,-0.845265,-0.664505,1.441345,-0.880028,-0.880916,-0.919520,3.0,3.0,3.0
4,1.250167,-1.127037,1.441345,1.282427,1.340698,1.165523,3.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...
129688,-1.039686,-0.016960,-0.642431,-1.267468,-1.243629,-1.238068,4.0,1.0,2.0
129689,0.990939,0.168053,-0.642431,1.048161,1.114003,1.078646,4.0,3.0,1.0
129690,0.450879,1.555648,-0.642431,0.516557,0.588069,0.702180,1.0,3.0,2.0
129691,0.666903,-1.774581,1.441345,0.867956,0.950782,0.687701,4.0,3.0,4.0


In [168]:
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())
X_test

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.564434,-0.942024,-0.642431,-0.429517,-0.463797,-0.499615,3.0,1.0,3.0
1,-0.175590,1.000610,-0.121487,-0.042077,-0.028542,0.036125,2.0,4.0,2.0
2,-1.061289,0.260559,-0.121487,-1.303509,-1.298036,-1.267027,4.0,4.0,7.0
3,0.969336,-0.201973,1.962289,1.048161,0.996121,0.977290,3.0,3.0,3.0
4,-0.931674,-1.312049,0.399457,-1.006171,-0.989730,-1.064315,3.0,1.0,4.0
...,...,...,...,...,...,...,...,...,...
63875,2.632721,-2.884658,0.920401,2.300582,2.247479,1.875017,3.0,6.0,4.0
63876,-0.953277,0.075546,-1.163375,-1.060232,-1.044137,-1.049835,4.0,5.0,4.0
63877,-0.802060,0.445572,-0.121487,-0.825967,-0.817442,-0.789205,4.0,0.0,2.0
63878,-0.586036,-0.109467,-1.163375,-0.474568,-0.509136,-0.485136,4.0,1.0,5.0


Model Training -- with three models

In [260]:
"""Simple linear regression 
   Ridge Regression (L2 regularisation)
   Lassso Regression (L1 regularisation)
"""

'Simple linear regression \n   Ridge Regression (L2 regularisation)\n   Lassso Regression (L1 regularisation)\n'

Evaluating the model scores

In [261]:
import numpy as np

def evaluating_scores(true, predict):
    rmse = np.sqrt(mean_squared_error(true, predict))
    r2_square = r2_score(true, predict)
    mse = mean_squared_error(true, predict)
    mae = mean_absolute_error(true, predict)
    return mae, rmse, r2_square, mse


In [262]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet

models ={
    "Simple linear regression":LinearRegression(),
   "Ridge Regression (L2 regularisation)": Ridge(),
   "Lassso Regression (L1 regularisation)": Lasso(),
   "ElasticNet": ElasticNet()
    }


In [263]:
X_train.shape

(129693, 9)

In [264]:
y_train.shape

(129693, 1)

In [265]:
models.keys()
   

dict_keys(['Simple linear regression', 'Ridge Regression (L2 regularisation)', 'Lassso Regression (L1 regularisation)', 'ElasticNet'])

In [266]:
models.values()

dict_values([LinearRegression(), Ridge(), Lasso(), ElasticNet()])

In [267]:
list(models)

['Simple linear regression',
 'Ridge Regression (L2 regularisation)',
 'Lassso Regression (L1 regularisation)',
 'ElasticNet']

In [268]:
model_list=[]
r2_list=[]


In [ ]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    # print(model)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    print("training score", model.score(X_train, y_train))

    #validation part 
    mae, rmse, r2_square, mse = evaluating_scores(y_predict, y_test)
    print(model)
    model_list.append(model)
    
    print("Model Training Performance")
    print("RMSE:", rmse)
    print("MAE:", mae)
    print("MSE:", mse)
    print("R2Score:", r2_square*100)
    r2_list.append(r2_square)
    print('='*35)

    




training score 0.9366002598295858
LinearRegression()
Model Training Performance
RMSE: 1012.0684598433756
MAE: 673.1192939264239
MSE: 1024282.5674097425
R2Score: 93.25596331107432
training score 0.9366002505827449
Ridge()
Model Training Performance
RMSE: 1012.0707353570211
MAE: 673.1524393157647
MSE: 1024287.1733661015
R2Score: 93.25563626775532
training score 0.9365839594510476
Lasso()
Model Training Performance
RMSE: 1012.0668588177022
MAE: 674.3126578340965
MSE: 1024279.3267171306
R2Score: 93.24838155547523
training score 0.85810795523477
ElasticNet()
Model Training Performance
RMSE: 1511.6076266944901
MAE: 1049.7070318613507
MSE: 2284957.6170809492
R2Score: 78.6907412889136


In [271]:
model_list, r2_list

([LinearRegression(), Ridge(), Lasso(), ElasticNet()],
 [0.9325596331107433,
  0.9325563626775533,
  0.9324838155547523,
  0.786907412889136])

saving the model and testing with the new data set

In [314]:
import joblib  
best_model = None
best_r2 = -float("inf") # Start with the lowest possible R² score
print(best_r2)

for i in range(len(list(models))):
    model = list(models.values())[i]
    print(model)

    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)

     # Debugging: Print all scores returned
    mae, rmse, r2_square, mse = evaluating_scores(y_test, y_predict)  
    print(f"✅ Model: {model}, R² Score: {r2_square*100}")

    # ✅ Ensure it picks the highest R² score
    if r2_square > best_r2:
        best_r2 = r2_square
        best_model = model


# ✅ Save the best model
if best_model:
    joblib.dump(best_model, f"{best_model}.pkl")
    print(f"\n✅ Best Model Saved: {best_model} with R² Score: {best_r2*100}")


    



-inf
LinearRegression()
✅ Model: LinearRegression(), R² Score: 93.69455849250947
Ridge()
✅ Model: Ridge(), R² Score: 93.69453013843089
Lasso()
✅ Model: Lasso(), R² Score: 93.69457844208038
ElasticNet()
✅ Model: ElasticNet(), R² Score: 85.93389455213153

✅ Best Model Saved: Lasso() with R² Score: 93.69457844208038


Load the model

In [316]:
loaded_model = joblib.load("Lasso().pkl")
loaded_model

Lasso()